In [9]:
import re
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [10]:
#IPL prices loading
cricket_data = pd.read_csv('IPLPlayerAuctionData.csv')
ipl_prices = pd.read_csv('ipl_retention_prices.csv')

# Convert `Amount` to crores
cricket_data['Amount_Crores'] = cricket_data['Amount'] / 1e7

# Identify retention price columns dynamically
price_columns = [col for col in ipl_prices.columns if re.search(r'\d{4} Retention Price \(INR Crores\)', col)]

# Process `ipl_prices` to get average price per player over all years present in the dataset
ipl_prices['Average_Price_Crores'] = ipl_prices[price_columns].mean(axis=1, skipna=True)

# Merge both datasets on player name
cricket_data = cricket_data.rename(columns={'Player': 'Player_Name'})
ipl_prices = ipl_prices.rename(columns={'Player Name': 'Player_Name'})
merged_prices = pd.concat([cricket_data[['Player_Name', 'Amount_Crores']],
                           ipl_prices[['Player_Name', 'Average_Price_Crores']]], axis=0)
average_prices = merged_prices.groupby('Player_Name').max()

In [33]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
df = pd.read_csv('cricket_data_updated.csv')

# Remove rows where 'Year' is 'No stats'
df = df[df['Year'] != 'No stats']

# Clean data in 'Highest_Score' column by removing '*'
df['Highest_Score'] = df['Highest_Score'].str.replace('*', '', regex=False)

# Convert specified columns to numeric type
numeric_cols = ['Runs_Scored', 'Highest_Score', 'Batting_Average', 'Batting_Strike_Rate',
                'Centuries', 'Half_Centuries', 'Fours', 'Sixes', 'Wickets_Taken',
                'Bowling_Average', 'Economy_Rate', 'Bowling_Strike_Rate', 'Four_Wicket_Hauls',
                'Five_Wicket_Hauls', 'Catches_Taken', 'Stumpings']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Define weights for batting metrics
batting_weights = {
    'Runs_Scored': 0.20,
    'Highest_Score': 0.10,
    'Batting_Average': 0.20,
    'Batting_Strike_Rate': 0.25,
    'Centuries': 0.10,
    'Half_Centuries': 0.05,
    'Fours': 0.05,
    'Sixes': 0.05
}

# Function to calculate batting rating
def calculate_bat_rating(row):
    return sum(row[col] * weight for col, weight in batting_weights.items() if not pd.isnull(row[col]))

# Calculate raw batting ratings and create the 'Bat_Rating' column
df['Bat_Rating'] = df.apply(calculate_bat_rating, axis=1)

# Normalize Bat_Rating to range [0, 1]
scaler = MinMaxScaler()
df['Normalized_Bat_Rating'] = scaler.fit_transform(df[['Bat_Rating']]) * 100  # Convert to percentage

# Define weights for bowling metrics
bowling_weights = {
    'Wickets_Taken': 0.30,
    'Economy_Rate': 0.25,
    'Bowling_Average': 0.20,
    'Bowling_Strike_Rate': 0.15,
    'Four_Wicket_Hauls': 0.05,
    'Five_Wicket_Hauls': 0.05
}

# Function to calculate bowling rating
def calculate_bowling_rating(row):
    bowling_rating = 0
    for col, weight in bowling_weights.items():
        if not pd.isnull(row[col]):
            if col in ['Economy_Rate', 'Bowling_Average', 'Bowling_Strike_Rate']:
                bowling_rating += weight / (1 + row[col])
            else:
                bowling_rating += row[col] * weight
    return bowling_rating

# Calculate raw bowling ratings and create the 'Bowling_Rating' column
df['Bowling_Rating'] = df.apply(calculate_bowling_rating, axis=1)

# Normalize Bowling_Rating to range [0, 1]
df['Normalized_Bowling_Rating'] = scaler.fit_transform(df[['Bowling_Rating']]) * 100  # Convert to percentage

# Define weights for keeper metrics
keeper_weights = {
    'Runs_Scored': 0.20,
    'Catches_Taken': 0.40,
    'Stumpings': 0.40
}

# Function to calculate keeper rating, applied only if the player is a wicketkeeper
def calculate_keeper_rating(row):
    if row['Is_WicketKeeper'] == 1:
        return sum(row[col] * weight for col, weight in keeper_weights.items() if not pd.isnull(row[col]))
    else:
        return 0  # Non-wicketkeepers get a rating of 0 for keeper rating

# Calculate raw keeper ratings and create the 'Keeper_Rating' column
df['Keeper_Rating'] = df.apply(calculate_keeper_rating, axis=1)

# Normalize Keeper_Rating to range [0, 1]
df['Normalized_Keeper_Rating'] = scaler.fit_transform(df[['Keeper_Rating']]) * 100  # Convert to percentage

# Calculate All-Rounder Rating for players who meet the 20% difference condition
all_rounder_ratings = df[
    abs(df['Normalized_Bat_Rating'] - df['Normalized_Bowling_Rating']) <= 20
].copy()

# Calculate combined All-Rounder Rating based on both Batting and Bowling Ratings
all_rounder_ratings['All_Rounder_Rating'] = (
    all_rounder_ratings['Normalized_Bat_Rating'] * 0.5 +
    all_rounder_ratings['Normalized_Bowling_Rating'] * 0.5
)

# Calculate average ratings by player
average_bat_ratings = df.groupby('Player_Name')['Normalized_Bat_Rating'].mean().sort_values(ascending=False)
average_bowling_ratings = df.groupby('Player_Name')['Normalized_Bowling_Rating'].mean().sort_values(ascending=False)
# Select top wicketkeepers by filtering on 'Is_WicketKeeper'
average_keeper_ratings = (
    df[df['Is_WicketKeeper'] == 1]
    .groupby('Player_Name')['Normalized_Keeper_Rating']
    .mean()
    .sort_values(ascending=False)
)
average_all_rounder_ratings = all_rounder_ratings.groupby('Player_Name')['All_Rounder_Rating'].mean().sort_values(ascending=False)

# Team selection based on user input
num_bat = int(input("Enter the number of batsmen you want: "))
num_bowl = int(input("Enter the number of bowlers you want: "))
num_allrounder = int(input("Enter the number of all-rounders you want: "))
num_keeper = int(input("Enter the number of wicket-keepers you want: "))

# Initialize lists to hold selected players in each role
selected_batsmen = []
selected_bowlers = []
selected_keepers = []
selected_allrounders = []
selected_players = set()

# Helper function to add a player to the appropriate list if not already selected
def add_player(rating_list, num_needed, selected_list, selected_players):
    count = 0
    for player in rating_list.index:
        if player not in selected_players:
            selected_list.append(player)
            selected_players.add(player)
            count += 1
        if count == num_needed:
            break

# Allocate players based on the highest rating they have in any category
add_player(average_bat_ratings, num_bat, selected_batsmen, selected_players)
add_player(average_bowling_ratings, num_bowl, selected_bowlers, selected_players)
add_player(average_keeper_ratings, num_keeper, selected_keepers, selected_players)
add_player(average_all_rounder_ratings, num_allrounder, selected_allrounders, selected_players)

# Display the dream team without duplicates
print("\nYour Dream Team:\n")
print("Batsmen:")
print("\n".join(selected_batsmen))
print("\nBowlers:")
print("\n".join(selected_bowlers))
print("\nWicket Keepers:")
print("\n".join(selected_keepers))
print("\nAll Rounders:")
print("\n".join(selected_allrounders))


Enter the number of batsmen you want: 5
Enter the number of bowlers you want: 4
Enter the number of all-rounders you want: 2
Enter the number of wicket-keepers you want: 6

Your Dream Team:

Batsmen:
Devon Conway
David Warner
Virat Kohli
Shubman Gill
Kyle Mayers

Bowlers:
Rashid Khan
Mukesh Choudhary
Noor Ahmad
Yuzvendra Chahal

Wicket Keepers:
KL Rahul
Rishabh Pant
Jos Buttler
Sanju Samson
MS Dhoni
Dinesh Karthik

All Rounders:
Hardik Pandya
Sam Curran


In [25]:
# Function to select dream team with budget
def calculate_price_multiplier(rating, avg_rating, std_dev):
    return 1 + (rating - avg_rating) * 0.1 / std_dev if std_dev != 0 else 1

def dream_team_with_budget(df, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
                           average_all_rounder_ratings, average_prices):

    total_budget = float(input("Enter your total budget in crores: "))
    bat_percent = float(input("Enter percentage of budget for batsmen: ")) / 100
    bowl_percent = float(input("Enter percentage of budget for bowlers: ")) / 100
    keeper_percent = float(input("Enter percentage of budget for keepers: ")) / 100
    allrounder_percent = float(input("Enter percentage of budget for all-rounders: ")) / 100

    budget_batsmen = total_budget * bat_percent
    budget_bowlers = total_budget * bowl_percent
    budget_keepers = total_budget * keeper_percent
    budget_allrounders = total_budget * allrounder_percent

    dream_team = {"Batsmen": [], "Bowlers": [], "Keepers": [], "All-Rounders": []}

    # Function to select players within budget
    def select_within_budget(ratings, budget, role):
        avg_rating = ratings.mean()
        std_dev = ratings.std()
        players_selected = []
        total_cost = 0
        for player, rating in ratings.items():
            avg_price = average_prices.loc[player, 'Average_Price_Crores'] if player in average_prices.index else None
            if avg_price is None:
                print(f"Player {player} not in dataset; moving to the next player.")
                continue
            multiplier = calculate_price_multiplier(rating, avg_rating, std_dev)
            predicted_price = avg_price * multiplier
            if total_cost + predicted_price <= budget:
                players_selected.append((player, predicted_price))
                total_cost += predicted_price
            elif len(players_selected) > 0:
                last_player, last_price = players_selected[-1]
                if total_cost - last_price + predicted_price <= budget:
                    players_selected[-1] = (player, predicted_price)
                    total_cost = total_cost - last_price + predicted_price
                    print(f"Replaced {last_player} with {player} to fit budget in {role}.")
        return players_selected

    # Select players for each role within the budget
    dream_team["Batsmen"] = select_within_budget(average_bat_ratings, budget_batsmen, "Batsmen")
    dream_team["Bowlers"] = select_within_budget(average_bowling_ratings, budget_bowlers, "Bowlers")
    dream_team["Keepers"] = select_within_budget(average_keeper_ratings, budget_keepers, "Keepers")
    dream_team["All-Rounders"] = select_within_budget(average_all_rounder_ratings, budget_allrounders, "All-Rounders")

    # Print the final dream team with predicted prices
    print("\nFinal Dream Team with Predicted Prices:\n")
    for role, players in dream_team.items():
        print(f"\n{role}:")
        for player, price in players:
            print(f"{player:<20} Predicted Price: {price:.2f} crores")

# Call the function with ratings
dream_team_with_budget(df, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
                       average_all_rounder_ratings, average_prices)


Enter your total budget in crores: 300
Enter percentage of budget for batsmen: 40
Enter percentage of budget for bowlers: 30
Enter percentage of budget for keepers: 10
Enter percentage of budget for all-rounders: 20
Player Faf du Plessis not in dataset; moving to the next player.
Replaced Rohit Sharma with MS Dhoni to fit budget in Batsmen.
Replaced Suryakumar Yadav with Shreyas Iyer to fit budget in Batsmen.
Player Nehal Wadhera not in dataset; moving to the next player.
Player Quinton de Kock not in dataset; moving to the next player.
Player Cameron Green not in dataset; moving to the next player.
Player Rachin Ravindra not in dataset; moving to the next player.
Player Phil Salt not in dataset; moving to the next player.
Player Abhinav Manohar not in dataset; moving to the next player.
Player Abishek Porel not in dataset; moving to the next player.
Player Naman Dhir not in dataset; moving to the next player.
Player K.S Bharat not in dataset; moving to the next player.
Player Rilee Ro

In [29]:
def calculate_price_multiplier(rating, avg_rating, std_dev):
    return 1 + (rating - avg_rating) * 0.1 / std_dev if std_dev != 0 else 1

def dream_team_with_budget(df, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
                           average_all_rounder_ratings, average_prices):

    total_budget = float(input("Enter your total budget in crores: "))
    bat_percent = float(input("Enter percentage of budget for batsmen: ")) / 100
    bowl_percent = float(input("Enter percentage of budget for bowlers: ")) / 100
    keeper_percent = float(input("Enter percentage of budget for keepers: ")) / 100
    allrounder_percent = float(input("Enter percentage of budget for all-rounders: ")) / 100

    budget_batsmen = total_budget * bat_percent
    budget_bowlers = total_budget * bowl_percent
    budget_keepers = total_budget * keeper_percent
    budget_allrounders = total_budget * allrounder_percent

    dream_team = {"Batsmen": [], "Bowlers": [], "Keepers": [], "All-Rounders": []}
    selected_players = set()  # Track selected players across roles

    # Define player limits for each role
    role_limits = {
        "Batsmen": 6,
        "Bowlers": 5,
        "Keepers": 2,
        "All-Rounders": 3
    }

    # Function to select players within budget and by role limit
    def select_within_budget(ratings, budget, role):
        avg_rating = ratings.mean()
        std_dev = ratings.std()
        players_selected = []
        total_cost = 0

        # Sort players by rating to prioritize higher-rated players
        sorted_ratings = dict(sorted(ratings.items(), key=lambda x: x[1], reverse=True))

        for player, rating in sorted_ratings.items():
            # Skip if player is already selected in another role
            if player in selected_players:
                continue

            avg_price = average_prices.loc[player, 'Average_Price_Crores'] if player in average_prices.index else None
            if avg_price is None:
                print(f"Player {player} not in dataset; moving to the next player.")
                continue

            multiplier = calculate_price_multiplier(rating, avg_rating, std_dev)
            predicted_price = avg_price * multiplier

            if total_cost + predicted_price <= budget and len(players_selected) < role_limits[role]:
                players_selected.append((player, predicted_price))
                selected_players.add(player)  # Add to selected players set
                total_cost += predicted_price

            # Stop selection if role limit is reached
            if len(players_selected) == role_limits[role]:
                break

        return players_selected

    # Select players for each role within the budget and limits
    # Process keepers first, as they're usually the most specialized
    dream_team["Keepers"] = select_within_budget(average_keeper_ratings, budget_keepers, "Keepers")
    dream_team["Batsmen"] = select_within_budget(average_bat_ratings, budget_batsmen, "Batsmen")
    dream_team["Bowlers"] = select_within_budget(average_bowling_ratings, budget_bowlers, "Bowlers")
    dream_team["All-Rounders"] = select_within_budget(average_all_rounder_ratings, budget_allrounders, "All-Rounders")

    # Print the final dream team with predicted prices
    print("\nFinal Dream Team with Predicted Prices:\n")
    total_team_cost = 0
    for role, players in dream_team.items():
        print(f"\n{role}:")
        role_cost = 0
        for player, price in players:
            print(f"{player:<20} Predicted Price: {price:.2f} crores")
            role_cost += price
            total_team_cost += price
        print(f"Total {role} cost: {role_cost:.2f} crores")

    print(f"\nTotal team cost: {total_team_cost:.2f} crores")
    print(f"Remaining budget: {(total_budget - total_team_cost):.2f} crores")

# Call the function with ratings
dream_team_with_budget(df, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
                       average_all_rounder_ratings, average_prices)


Enter your total budget in crores: 150
Enter percentage of budget for batsmen: 40
Enter percentage of budget for bowlers: 30
Enter percentage of budget for keepers: 10
Enter percentage of budget for all-rounders: 20
Player Quinton de Kock not in dataset; moving to the next player.
Player Faf du Plessis not in dataset; moving to the next player.
Player Nehal Wadhera not in dataset; moving to the next player.
Player Quinton de Kock not in dataset; moving to the next player.
Player Cameron Green not in dataset; moving to the next player.
Player Rachin Ravindra not in dataset; moving to the next player.
Player Phil Salt not in dataset; moving to the next player.
Player Rashid Khan not in dataset; moving to the next player.
Player Akash Madhwal not in dataset; moving to the next player.
Player Sunil Narine not in dataset; moving to the next player.
Player Matheesha Pathirana not in dataset; moving to the next player.
Player Umran Malik not in dataset; moving to the next player.
Player Vysha

2

In [41]:
import pandas as pd

# Load the CSV files
cricket_data = pd.read_csv('IPLPlayerAuctionData.csv')
ipl_prices = pd.read_csv('ipl_retention_prices.csv')

# Renaming 'Player Name' in ipl_prices to 'Player' for consistency
ipl_prices.rename(columns={'Player Name': 'Player'}, inplace=True)

# Convert 'Amount' in cricket_data from INR to crores (assuming 'Amount' is in INR)
cricket_data['Amount_Crores'] = cricket_data['Amount'] / 1e7  # Convert to crores

# Calculate the average price for each player in ipl_prices across available years
price_columns = [col for col in ipl_prices.columns if 'Retention Price' in col]
ipl_prices['Average_Price_Crores'] = ipl_prices[price_columns].mean(axis=1)

# Function to calculate the average price for a player from both datasets
def get_average_price(player, df_cricket, df_prices):
    # Check if the player has an average price in the `ipl_prices` dataset
    price_row = df_prices[df_prices['Player'] == player]['Average_Price_Crores']
    if not price_row.empty:
        return price_row.values[0]

    # If not found in `ipl_prices`, check the `cricket_data` for yearly auction prices
    yearly_prices = df_cricket[df_cricket['Player'] == player]['Amount_Crores']
    if not yearly_prices.empty:
        return yearly_prices.mean()  # Average over years played

    # If the player is not found in either dataset, return None
    print(f"Warning: {player} not found in either dataset.")
    return None

def calculate_price_multiplier(rating, avg_rating, std_dev):
    return 1 + (rating - avg_rating) * 0.1 / std_dev if std_dev != 0 else 1

# Define the main function to create a dream team with budget constraints
def dream_team_with_budget(df, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
                           average_all_rounder_ratings):
    total_budget = float(input("Enter your total budget in crores: "))
    bat_percent = float(input("Enter percentage of budget for batsmen: ")) / 100
    bowl_percent = float(input("Enter percentage of budget for bowlers: ")) / 100
    keeper_percent = float(input("Enter percentage of budget for keepers: ")) / 100
    allrounder_percent = float(input("Enter percentage of budget for all-rounders: ")) / 100

    budget_batsmen = total_budget * bat_percent
    budget_bowlers = total_budget * bowl_percent
    budget_keepers = total_budget * keeper_percent
    budget_allrounders = total_budget * allrounder_percent

    dream_team = {"Batsmen": [], "Bowlers": [], "Keepers": [], "All-Rounders": []}
    selected_players = set()

    # Define player limits for each role
    role_limits = {
        "Batsmen": 6,
        "Bowlers": 5,
        "Keepers": 2,
        "All-Rounders": 3
    }

    # Function to select players within budget for a given role
    def select_within_budget(ratings, budget, role):
        avg_rating = ratings.mean()
        std_dev = ratings.std()
        players_selected = []
        total_cost = 0

        sorted_ratings = dict(sorted(ratings.items(), key=lambda x: x[1], reverse=True))

        for player, rating in sorted_ratings.items():
            if player in selected_players:
                continue

            avg_price = get_average_price(player, df, ipl_prices)
            if avg_price is None:
                print(f"Skipping {player} due to missing price information.")
                continue

            multiplier = calculate_price_multiplier(rating, avg_rating, std_dev)
            predicted_price = avg_price * multiplier

            if total_cost + predicted_price <= budget and len(players_selected) < role_limits[role]:
                players_selected.append((player, predicted_price))
                selected_players.add(player)
                total_cost += predicted_price

            if len(players_selected) == role_limits[role]:
                break

        return players_selected

    # Select players for each role within budget
    dream_team["Keepers"] = select_within_budget(average_keeper_ratings, budget_keepers, "Keepers")
    dream_team["Batsmen"] = select_within_budget(average_bat_ratings, budget_batsmen, "Batsmen")
    dream_team["Bowlers"] = select_within_budget(average_bowling_ratings, budget_bowlers, "Bowlers")
    dream_team["All-Rounders"] = select_within_budget(average_all_rounder_ratings, budget_allrounders, "All-Rounders")

    # Print the final dream team with predicted prices
    print("\nFinal Dream Team with Predicted Prices:\n")
    total_team_cost = 0
    for role, players in dream_team.items():
        print(f"\n{role}:")
        role_cost = 0
        for player, price in players:
            print(f"{player:<20} Predicted Price: {price:.2f} crores")
            role_cost += price
            total_team_cost += price
        print(f"Total {role} cost: {role_cost:.2f} crores")

    print(f"\nTotal team cost: {total_team_cost:.2f} crores")
    print(f"Remaining budget: {(total_budget - total_team_cost):.2f} crores")

# Example call to the function (rating dictionaries need to be provided in actual use)
# dream_team_with_budget(cricket_data, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
#                        average_all_rounder_ratings)

In [43]:
import pandas as pd

# Load the CSV files
cricket_data = pd.read_csv('IPLPlayerAuctionData.csv')
ipl_prices = pd.read_csv('ipl_retention_prices.csv')

# Renaming 'Player Name' in ipl_prices to 'Player' for consistency
ipl_prices.rename(columns={'Player Name': 'Player'}, inplace=True)

# Convert 'Amount' in cricket_data from INR to crores (assuming 'Amount' is in INR)
cricket_data['Amount_Crores'] = cricket_data['Amount'] / 1e7  # Convert to crores

# Calculate the average price for each player in ipl_prices across available years
price_columns = [col for col in ipl_prices.columns if 'Retention Price' in col]
ipl_prices['Average_Price_Crores'] = ipl_prices[price_columns].mean(axis=1)

# Function to calculate the average price for a player from both datasets
def get_average_price(player, df_cricket, df_prices):
    # Check if the player has an average price in the `ipl_prices` dataset
    price_row = df_prices[df_prices['Player'] == player]['Average_Price_Crores']
    if not price_row.empty:
        return price_row.values[0]

    # If not found in `ipl_prices`, check the `cricket_data` for yearly auction prices
    yearly_prices = df_cricket[df_cricket['Player'] == player]['Amount_Crores']
    if not yearly_prices.empty:
        return yearly_prices.mean()  # Average over years played

    # If the player is not found in either dataset, return None
    print(f"Warning: {player} not found in either dataset.")
    return None

def calculate_price_multiplier(rating, avg_rating, std_dev):
    return 1 + (rating - avg_rating) * 0.1 / std_dev if std_dev != 0 else 1

# Define the main function to create a dream team with budget constraints
def dream_team_with_budget(df, average_bat_ratings, average_bowling_ratings, average_keeper_ratings,
                           average_all_rounder_ratings):
    # Hardcoded budget and budget allocations for testing
    total_budget = 100  # Total budget in crores
    bat_percent = 0.25  # 25% budget allocation for batsmen
    bowl_percent = 0.25  # 25% budget allocation for bowlers
    keeper_percent = 0.25  # 25% budget allocation for keepers
    allrounder_percent = 0.25  # 25% budget allocation for all-rounders

    # Calculate allocated budgets for each role
    budget_batsmen = total_budget * bat_percent
    budget_bowlers = total_budget * bowl_percent
    budget_keepers = total_budget * keeper_percent
    budget_allrounders = total_budget * allrounder_percent

    dream_team = {"Batsmen": [], "Bowlers": [], "Keepers": [], "All-Rounders": []}
    selected_players = set()

    # Define player limits for each role
    role_limits = {
        "Batsmen": 6,
        "Bowlers": 5,
        "Keepers": 2,
        "All-Rounders": 3
    }

    # Function to select players within budget for a given role
    def select_within_budget(ratings, budget, role):
        avg_rating = pd.Series(ratings).mean()
        std_dev = pd.Series(ratings).std()
        players_selected = []
        total_cost = 0

        sorted_ratings = dict(sorted(ratings.items(), key=lambda x: x[1], reverse=True))

        for player, rating in sorted_ratings.items():
            if player in selected_players:
                continue

            avg_price = get_average_price(player, df, ipl_prices)
            if avg_price is None:
                print(f"Skipping {player} due to missing price information.")
                continue

            multiplier = calculate_price_multiplier(rating, avg_rating, std_dev)
            predicted_price = avg_price * multiplier

            if total_cost + predicted_price <= budget and len(players_selected) < role_limits[role]:
                players_selected.append((player, predicted_price))
                selected_players.add(player)
                total_cost += predicted_price

            if len(players_selected) == role_limits[role]:
                break

        return players_selected

    # Select players for each role within budget
    dream_team["Keepers"] = select_within_budget(average_keeper_ratings, budget_keepers, "Keepers")
    dream_team["Batsmen"] = select_within_budget(average_bat_ratings, budget_batsmen, "Batsmen")
    dream_team["Bowlers"] = select_within_budget(average_bowling_ratings, budget_bowlers, "Bowlers")
    dream_team["All-Rounders"] = select_within_budget(average_all_rounder_ratings, budget_allrounders, "All-Rounders")

    # Print the final dream team with predicted prices
    print("\nFinal Dream Team with Predicted Prices:\n")
    total_team_cost = 0
    for role, players in dream_team.items():
        print(f"\n{role}:")
        role_cost = 0
        for player, price in players:
            print(f"{player:<20} Predicted Price: {price:.2f} crores")
            role_cost += price
            total_team_cost += price
        print(f"Total {role} cost: {role_cost:.2f} crores")

    print(f"\nTotal team cost: {total_team_cost:.2f} crores")
    print(f"Remaining budget: {(total_budget - total_team_cost):.2f} crores")

# Example rating dictionaries for testing
average_bat_ratings = {"Player1": 85, "Player2": 90, "Player3": 78}
average_bowling_ratings = {"Player4": 88, "Player5": 82}
average_keeper_ratings = {"Player6": 91, "Player7": 85}
average_all_rounder_ratings = {"Player8": 89, "Player9": 86}

# Call the function with hardcoded budgets and rating dictionaries
dream_team_with_budget(cricket_data, average_bat_ratings, average_bowling_ratings,
                       average_keeper_ratings, average_all_rounder_ratings)


Skipping Player6 due to missing price information.
Skipping Player7 due to missing price information.
Skipping Player2 due to missing price information.
Skipping Player1 due to missing price information.
Skipping Player3 due to missing price information.
Skipping Player4 due to missing price information.
Skipping Player5 due to missing price information.
Skipping Player8 due to missing price information.
Skipping Player9 due to missing price information.

Final Dream Team with Predicted Prices:


Batsmen:
Total Batsmen cost: 0.00 crores

Bowlers:
Total Bowlers cost: 0.00 crores

Keepers:
Total Keepers cost: 0.00 crores

All-Rounders:
Total All-Rounders cost: 0.00 crores

Total team cost: 0.00 crores
Remaining budget: 100.00 crores
